# 概要
MNISTの分類を、全結合型ニューラルネットワークで行う。
* 第9回実習と同じモデル
* 入力成分の順番をランダムに入れ替え

# 乱数シードの固定

In [ ]:
import random
import numpy as np
import os

np.random.seed(0)
random.seed(0)
os.environ["PYTHONHASHSEED"] = str(0)

# 関数 save_fig：図の保存用

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

def save_fig(plt, file_prefix):
    if file_prefix == '':
        return
    
    parent = os.path.dirname(os.path.abspath(file_prefix))
    os.makedirs(parent, exist_ok=True)
    plt.savefig(f'{file_prefix}.pdf', transparent=True, bbox_inches='tight', pad_inches = 0)
    plt.savefig(f'{file_prefix}.png', transparent=True, dpi=300, bbox_inches='tight', pad_inches = 0)

# 関数 show_prediction：MNISTに対する予測結果の可視化

In [ ]:
def show_prediction(x, y_true, y_pred, ids, file_prefix=''):
    '''
    手書き数字の認識結果を表示する。
    
    Parameters
    ------------
    model : Sequential
        ニューラルネットワーク・モデル
    '''

    img = x[ids].reshape((len(ids), nx, ny))

    labels = y_true[ids]
    preds = y_pred[ids]

    # AIが認識した結果を画像と一緒に提示する
    plt.figure(2, figsize=(12, 8))
    plt.gray()
    for i in range(len(ids)):
        plt.subplot(8, 12, i + 1)
        plt.pcolor(img[i])
        plt.text(22, 25.5, "%d" % preds[i], fontsize=12, color='yellow')
        if preds[i] != labels[i]:
            plt.plot([0, nx - 1], [1, 1], color='red', linewidth=5)
        
        plt.xlim(0, nx - 1)
        plt.ylim(ny - 1, 0)
        plt.xticks([], '')
        plt.yticks([], '')
    
    plt.tight_layout()
    save_fig(plt, file_prefix=file_prefix)
    plt.show()    

# 関数 show_mnist_images：MNISTの表示

In [ ]:
def show_mnist_images(x, y_true, ids, ny=28, nx=28, file_prefix=''):
    '''
    カラー画像のサンプルを表示する。
    
    Parameters
    ------------
    x: ndarray
        カラー画像。形状(N)。N:データ数
    y_true: ndarray
        データxに対応するクラスラベルの正解インデックス。形状(N)
    labels: list
        クラスラベル
    ids: ndarray
        表示するデータのインデックス
    ny: int
        縦方向画素数
    nx: int
        横方向画素数
    file_prefix: str
        保存時のファイルプレフィックス

    Returns
    -------
    なし
    '''

    img = x[ids].reshape((len(ids), nx, ny))

    trues = y_true[ids]

    plt.figure(figsize=(24, 16))
    for i in range(len(ids)):
        plt.subplot(8, 12, i + 1)
        plt.imshow(img[i], cmap='gray')
        t = trues[i]
        plt.title(f'{t}', fontsize=12)       
        plt.axis('off')

    plt.tight_layout()
    save_fig(plt, file_prefix=file_prefix)
    plt.show()    

# 関数 evaluate：性能評価関数

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
import seaborn as sns

def evalulate(x, y_true, y_pred, file_prefix=''):
    
    cm = confusion_matrix(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average=None)
    recall = recall_score(y_true, y_pred, average=None)
    print('正解率')
    print(f' {accuracy:.2f}')
    
    class_labels = []

    num_classes = np.max(y_true) + 1
    for i in range(num_classes):
        class_labels.append(f'{i:4d}')
    
    precision_str = []
    recall_str = []
    for i in range(num_classes):
        precision_str.append(f'{precision[i]:.2f}')
        recall_str.append(f'{recall[i]:.2f}')

    print('精度')
    print(' ' + ' '.join(class_labels))
    print(' ' + ' '.join(precision_str))
                             
    print('再現率')
    print(' ' + ' '.join(class_labels))
    print(' ' + ' '.join(recall_str))

    plt.figure(figsize = (10,7))
    sns.heatmap(cm, annot=True, fmt='3d', square=True, cmap='hot')
    plt.tight_layout()
    save_fig(plt, file_prefix=file_prefix)
    plt.show()

# 関数 show_history：学習過程の可視化

In [ ]:
def show_history(history, file_prefix=''):
    plt.figure(figsize=(16, 8))
    plt.subplots_adjust(wspace=0.2)
    
    # 学習曲線の表示
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], 'black', label='Training')
    plt.plot(history.history['val_loss'], 'cornflowerblue', label='Test')
    plt.legend(fontsize=16)
    plt.xlabel('Epochs', fontsize=16)
    plt.ylabel('Value', fontsize=16)
    plt.title('Loss', fontsize=16)
    plt.ylim(0, )
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)   
    plt.grid(True)
    
    # 精度表示
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], 'black', label='Training')
    plt.plot(history.history['val_accuracy'], 'cornflowerblue', label='Test')
    plt.legend(fontsize=16)
    plt.xlabel('Epochs', fontsize=16)
    plt.ylabel('Value', fontsize=16)
    plt.title('Accuracy', fontsize=16)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.ylim(0, 1.2)
    plt.grid(True)

    save_fig(plt, file_prefix)
    
    plt.show()

# 関数 load_mnist：教師データを準備する関数

In [ ]:
from tensorflow.keras.datasets import mnist
import numpy as np
import random

def load_mnist():
    """
    手書き数字データセットMNISTを全結合型ニューラルネットワークの教師データとして用意する。

    Returns
    -------
    x_train: ndarray
        訓練データの画像配列。形状(60000, 784)
    y_train: ndarray
        訓練データのクラスラベル。one-hotベクトル化済み。形状(60000, 10)
    x_test: ndarray
        テストデータの画像配列。形状(10000, 784)
    y_test: ndarray
        テストデータのクラスラベル。one-hotベクトル化済み。形状(10000, 10)
    ny: int
        画像の縦方向画素数
    nx: int
        画像の横方向画素数
    num_classes: int
        クラス数
    """
    # MNISTデータセットをTensorFlowのサイトからロード
    # (訓練データ画像, 訓練データ・クラスラベル), (テストデータ画像, テストデータ・クラスラベル)
    (x_train0, y_train0), (x_test0, y_test0) = mnist.load_data()

    # 訓練データの準備
    print('訓練データ')
    print('画像枚数 =', len(x_train0))

    # 先頭の画像を取り出して、形状を取得
    ny, nx = x_train0[0].shape
    print('画像の縦方向画素数 =', ny)
    print('画像の横方向画素数 =', nx)

    # 訓練データ数 60000
    # 1つの画像は 28 x 28 = 784画素
    # 0から59999の各行には1個の画像が格納される。
    # 全結合層へ入力するため、各画像をny * nx個の成分を持つ1次元配列へ変換。
    x_train = x_train0.reshape(len(x_train0), ny * nx)

    x_train = x_train.astype('float32')   # 要素値の型をfloat32へ変更
    # 画素値の最大値255を使い、画像の値を0以上1以下の範囲にする（正規化）
    x_train = x_train / 255               

    # クラス数。0から9の数字なので、10個
    num_classes = np.max(y_train0) + 1
    print('クラス数 =', num_classes)

    # one-hotベクトル化
    # なぜこう書けるのか、わからない人は実習資料を参照
    mat = np.identity(num_classes)
    y_train = mat[y_train0]
    print('クラスラベルの形状 =', y_train.shape)

    # テストデータの準備
    # テストデータ数 10000
    print('\nテストデータ')
    print('画像枚数 =', len(x_test0))

    # テストデータ数 10000
    # 1つの画像は 28 x 28 = 784画素
    # 0から9999の各行には1個の画像が格納される。
    # 全結合層へ入力するため、各画像をny * nx個の成分を持つ1次元配列へ変換。
    x_test = x_test0.reshape(len(x_test0), ny * nx)
    x_test = x_test.astype('float32')  # 要素値の型をfloat32へ変更
    # 画素値の最大値255を使い、画像の値を0以上1以下の範囲にする（正規化）
    x_test = x_test / 255
    
    y_test = mat[y_test0]              # one-hotベクトル化

    return x_train, y_train, x_test, y_test, ny, nx, num_classes

# 実習9.1：10層の中間層（活性化関数ReLU）を持つmodel2を構築

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import plot_model

# MNIST教師データを再生成
# 戻り値の各変数の役割は関数定義を参照。
x_train, y_train, x_test, y_test, ny, nx, num_classes = load_mnist()

ids = range(96) # 可視化するデータの番号0から95
print('成分の入れ替え前')
show_mnist_images(x_train, np.argmax(y_train, axis=1), ids, file_prefix='before_shuffle')

# 画像を1次元配列とみなして、成分を入れ替える
random_idx = np.arange(nx * ny)
random.shuffle(random_idx)
x_train = x_train[:, random_idx]
x_test = x_test[:, random_idx]

print('成分の入れ替え後：学習に使用')
show_mnist_images(x_train, np.argmax(y_train, axis=1), ids, file_prefix='after_shuffle')

# ネットワークのパラメータ（重みとバイアス）を初期化する際の乱数シードを固定
tf.random.set_seed(0)

n_hidden = 10 # 中間層数

# ネットワークモデルを生成。層（レイヤー）の容器と考える。
model1 = Sequential()

# 最初の中間層を追加
model1.add(Dense(input_dim=ny*nx,   # 入力層のニューロン数(入力データの成分数)
                 units=10,          # 中間層1のニューロン数
                 activation='relu', # 活性化関数ReLU
                 name='hidden1'))   # 中間層1の名前

# 2から9番目の中間層を追加
for i in range(2, n_hidden + 1):
    model1.add(Dense(units=10,           # 中間層iのニューロン数
                     activation='relu',  # 活性化関数ReLU
                     name=f'hidden{i}')) # 中間層iの名前

# 出力層を追加
model1.add(Dense(units=10,              # 出力層のニューロン数(クラス数)
                 activation='softmax',  # 活性化関数
                 name='output'))        # 出力層の名前

model1.compile(optimizer='Adam',                # パラメータの最適化手法にAdamを指定
               loss='categorical_crossentropy', # 損失関数
               metrics=['accuracy'])            # 損失の他にモニターする指標として正解率を指定

# モデルの要約情報を表示
model1.summary()

# 実習9.2：model2の学習

In [ ]:
# 学習開始。終了後、学習の履歴がhistory2に代入される。
history1 = model1.fit(x_train,  # 訓練データ：入力画像
                      y_train,  # 訓練データ：クラスラベル
                      epochs=500,  # エポック数
                      batch_size=1000, # バッチサイズ
                      shuffle=True,    # 学習時にデータの並びをシャッフル
                      validation_data=(x_test, y_test)) # 学習過程での汎化能力の推定のためテストデータを指定

# 実習9.3：model1の学習の履歴と汎化能力の推定

In [ ]:
# 学習の履歴（損失と正解率の変化）を表示
show_history(history1, file_prefix='9.3_loss_and_acc')

# 学習したモデルを使い、テストデータのクラスラベルを予測
y_pred = model1.predict(x_test)

# 各データについて、最大の予測確率をもつクラスラベルを取得
y_pred = np.argmax(y_pred, axis=1)

# 正解（one-hotベクトル）をクラスラベルに戻す
y_true = np.argmax(y_test, axis=1)

# 混同行列、正解率、精度、再現率を評価
evalulate(x_test, y_true, y_pred, file_prefix='9.3_cm')

# 予測結果の一部を可視化する
ids = range(96) # 可視化するデータの番号0から95

# データの番号0から95について、予測結果を可視化
show_prediction(x_test, y_true, y_pred, ids, file_prefix='9.3_digits')